In [1]:
#fundamental tools for the game environment model
import numpy as np
import math
import random
import itertools

# for visualations
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import exploreModel
import explore_Viewer

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
test = np.arange(100).reshape(10,10)
test = test/test.max()*255
test = test.astype("int")

In [4]:
env = exploreModel.exploreGame()

In [5]:
fr = env.generate_frame()

In [6]:
fr.reshape(-1,3)

array([[  0., 255.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       ...,
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.]])

In [7]:
viewer = explore_Viewer.exploreViewer(env.generate_frame(), env.get_trace())

In [8]:
viewer.start()